In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# import warnings
import warnings
# filter warnings
warnings.filterwarnings('ignore')
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical

from keras.models import Sequential 
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop,Adam
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

print('TensorFlow version:', tf.__version__)
print('Keras version:', keras.__version__)

['test', 'sample_submission.csv', 'train.csv', 'train']


Using TensorFlow backend.


NameError: name 'tf' is not defined

In [2]:
import os
print(os.listdir("../input"))

['test', 'sample_submission.csv', 'train.csv', 'train']


In [3]:
TRAIN_IMAGE_PATH = "../input/train/"
TEST_IMAGE_PATH = "../input/test/"
TRAINING_DATA='../input/train.csv'
IMG_SIZE = 100

In [4]:
df_train = pd.read_csv(TRAINING_DATA)
#string to unique int
#set unique int value for each unique classes sring.. string to int
unique_calsses_value = np.unique(df_train[['Id']].values)
print(unique_calsses_value)
unique_classes_id_dict = {}
unique_id_classes_dict = {}
for i in range(len(unique_calsses_value)):
    unique_classes_id_dict[unique_calsses_value[i]] = i
    unique_id_classes_dict[i] = unique_calsses_value[i]

['new_whale' 'w_0003639' 'w_0003c59' ... 'w_ffa6d42' 'w_ffcf5fe'
 'w_ffe8693']


In [5]:
#add new class_id col in df_train df
df_train['classes_id'] = df_train.apply (lambda row: unique_classes_id_dict.get(row['Id']),axis=1)
df_train.head(5)

,Image,Id,classes_id
0,0000e88ab.jpg,w_f48451c,4786
1,0001f9222.jpg,w_c3d896a,3808
2,00029d126.jpg,w_20df2c5,662
3,00050a15a.jpg,new_whale,0
4,0005c1ef8.jpg,new_whale,0


In [6]:
def prepareImages(train, shape, path):    
    x_train = np.zeros((shape, 100, 100, 3))
    count = 0
    
    for fig in train['Image']:
        
        #load images into images of size 100x100x3
        img = image.load_img("../input/"+path+"/"+fig, target_size=(100, 100, 3))
        x = image.img_to_array(img)
        x = preprocess_input(x)

        x_train[count] = x
        if (count%500 == 0):
            print("Processing image: ", count+1, ", ", fig)
        count += 1
    
    return x_train

In [7]:
# put labels into y_train variable
y_train = df_train["Id"]
# Drop 'Id' column
X_train = df_train.drop(labels = ["Id"], axis = 1)
y_train.head()

0    w_f48451c
1    w_c3d896a
2    w_20df2c5
3    new_whale
4    new_whale
Name: Id, dtype: object

In [8]:
x_train = prepareImages(df_train, df_train.shape[0], "train")

Processing image:  1 ,  0000e88ab.jpg
Processing image:  501 ,  04c72257b.jpg
Processing image:  1001 ,  09cacb84d.jpg
Processing image:  1501 ,  0ef961892.jpg
Processing image:  2001 ,  141b56a1a.jpg
Processing image:  2501 ,  199a417aa.jpg
Processing image:  3001 ,  1ec170983.jpg
Processing image:  3501 ,  23f084b93.jpg
Processing image:  4001 ,  29163ad0b.jpg
Processing image:  4501 ,  2e0fab120.jpg
Processing image:  5001 ,  3347515d9.jpg
Processing image:  5501 ,  3842d71dc.jpg
Processing image:  6001 ,  3d7f4c7d5.jpg
Processing image:  6501 ,  425f763ca.jpg
Processing image:  7001 ,  4714400cd.jpg
Processing image:  7501 ,  4c082fbdf.jpg
Processing image:  8001 ,  50c683e23.jpg
Processing image:  8501 ,  560d986ad.jpg
Processing image:  9001 ,  5b68c83ed.jpg
Processing image:  9501 ,  60410f111.jpg
Processing image:  10001 ,  654951f81.jpg
Processing image:  10501 ,  6a572256c.jpg
Processing image:  11001 ,  6f96f55b6.jpg
Processing image:  11501 ,  74da2b511.jpg
Processing image

In [9]:
x_train = x_train / 255.0
print("x_train shape: ",x_train.shape)

label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_train = to_categorical(y_train, num_classes = 5005)
print("y_train shape: ",y_train.shape)

x_train shape:  (25361, 100, 100, 3)
y_train shape:  (25361, 5005)


In [10]:
#CNN model
def cnn():
    model = Sequential()

    model.add(Conv2D(filters = 16, kernel_size = (5,5), padding = 'Same', activation = 'relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(Conv2D(filters = 16, kernel_size = (5,5), padding = 'Same', activation = 'relu'))
    model.add(MaxPool2D(pool_size = (2,2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(filters = 32, kernel_size = (3,3), padding = 'Same', activation = 'relu'))
    model.add(Conv2D(filters = 32, kernel_size = (3,3), padding = 'Same', activation = 'relu'))
    model.add(MaxPool2D(pool_size = (2,2), strides=(2,2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = 'Same', activation = 'relu'))
    model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = 'Same', activation = 'relu'))
    model.add(MaxPool2D(pool_size = (2,2), strides=(2,2)))
    model.add(Dropout(0.25))

    # fully connected
    model.add(Flatten())
    model.add(Dense(256, activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Dense(y_train.shape[1], activation = "softmax"))
    model.summary()
    return model

In [11]:
model = cnn()
optimizer = Adam(lr = 0.001, beta_1 = 0.9, beta_2 = 0.999)
model.compile(optimizer = optimizer, loss = "categorical_crossentropy", metrics=["accuracy"])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 100, 100, 16)      1216      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 100, 100, 16)      6416      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 50, 50, 16)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 50, 50, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 50, 50, 32)        4640      
_________________________________________________________________
conv2d_4 (Conv2D)    

In [12]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)
epochs = 50
batch_size = 1000

In [13]:
history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, verbose=2, callbacks=[learning_rate_reduction])

Instructions for updating:
Use tf.cast instead.
Epoch 1/50
 - 25s - loss: 8.0136 - acc: 0.2185
Epoch 2/50
 - 17s - loss: 6.9430 - acc: 0.3046
Epoch 3/50
 - 17s - loss: 6.4376 - acc: 0.3263
Epoch 4/50
 - 17s - loss: 6.0647 - acc: 0.3402
Epoch 5/50
 - 17s - loss: 5.7679 - acc: 0.3448
Epoch 6/50
 - 17s - loss: 5.4928 - acc: 0.3511
Epoch 7/50
 - 17s - loss: 5.1903 - acc: 0.3658
Epoch 8/50
 - 17s - loss: 4.9352 - acc: 0.3749
Epoch 9/50
 - 17s - loss: 4.7260 - acc: 0.3801
Epoch 10/50
 - 17s - loss: 4.5619 - acc: 0.3815
Epoch 11/50
 - 17s - loss: 4.3860 - acc: 0.3836
Epoch 12/50
 - 17s - loss: 4.2155 - acc: 0.3854
Epoch 13/50
 - 17s - loss: 4.0400 - acc: 0.3894
Epoch 14/50
 - 17s - loss: 3.8580 - acc: 0.3919
Epoch 15/50
 - 17s - loss: 3.6680 - acc: 0.3996
Epoch 16/50
 - 17s - loss: 3.4734 - acc: 0.4082
Epoch 17/50
 - 17s - loss: 3.2828 - acc: 0.4181
Epoch 18/50
 - 17s - loss: 3.0797 - acc: 0.4320
Epoch 19/50
 - 17s - loss: 2.8643 - acc: 0.4488
Epoch 20/50
 - 17s - loss: 2.6515 - acc: 0.4687
E

In [14]:
test = os.listdir("../input/test/")
print(len(test))
col = ['Image']
test_data = pd.DataFrame(test, columns=col)
test_data['Id'] = ''
print(test_data.head())
x_test = prepareImages(test_data, test_data.shape[0], "test")
x_test /= 255

7960
           Image Id
0  009dd6c96.jpg   
1  2a73b60f7.jpg   
2  4d2f1d959.jpg   
3  0c9dd9352.jpg   
4  f6c4f44c5.jpg   
Processing image:  1 ,  009dd6c96.jpg
Processing image:  501 ,  e46beacae.jpg
Processing image:  1001 ,  092947f89.jpg
Processing image:  1501 ,  16af56ec3.jpg
Processing image:  2001 ,  4694081ba.jpg
Processing image:  2501 ,  af3d0515e.jpg
Processing image:  3001 ,  7670d7d5f.jpg
Processing image:  3501 ,  d60b30490.jpg
Processing image:  4001 ,  37bfa9a13.jpg
Processing image:  4501 ,  ad00da0c4.jpg
Processing image:  5001 ,  decc807d9.jpg
Processing image:  5501 ,  f6d9efbda.jpg
Processing image:  6001 ,  e48c9699b.jpg
Processing image:  6501 ,  ad0ac1e70.jpg
Processing image:  7001 ,  e9cabaa8c.jpg
Processing image:  7501 ,  b52b0306d.jpg


In [15]:
predictions = model.predict(np.array(x_test), verbose=1)

7960/7960 [==============================] - 2s 252us/step


In [16]:
for i, pred in enumerate(predictions):
    test_data.loc[i, 'Id'] = ' '.join(label_encoder.inverse_transform(pred.argsort()[-5:][::-1]))
test_data.head(10)
test_data.to_csv('submission_3.csv', index=False)